In [ ]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=cc4bf6d395f8793937ff0899a6f0aa905e80fd5ce0f8a21fed6b41cb401f24cb
  Stored in directory: /root/.cache/pip/wheels/2d/f0/a8/1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators


In [ ]:
%%writefile app.py

import numpy as np
import pickle
import streamlit as st
import math
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import io
import webbrowser
import tensorflow as tf

# Load the saved model
model = tf.keras.models.load_model('workout_model2.h5')

# Load the label names
with open('workout_label.txt', 'r') as f:
    labels = f.read().splitlines()

transformer = torchvision.transforms.Compose([
    transforms.Resize(size = (224, 224)),
    transforms.ToTensor(),
])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torch.load('model.pth')
model.eval()

dir_path= 'YogaData'

datafolder = torchvision.datasets.ImageFolder(dir_path, transform = transformer)
classes = datafolder.classes

@st.cache(suppress_st_warning=True)

# This is the main function in which we define our webpage
def main():
    
    options = ["Gym", "Yoga Pose"]
    selected_option = st.radio("Select an excercise", options)
    
    # Allow user to upload a file
    uploaded_file = st.file_uploader("Upload the image", type="jpg")

    # If a file was uploaded
    if uploaded_file is not None:
        # Display the image
        image = Image.open(io.BytesIO(uploaded_file.read()))
        st.write(type(image))
        st.image(image, caption='Uploaded Image')
        
    # When 'Predict' is clicked, make the prediction and store it
    result = ''
    if st.button("Find the excercise"):
        if(selected_option=='Yoga Pose'):
            transformed_img = transformer(image)
            transformed_img = transformed_img.unsqueeze(dim=0)
            transformed_img.to(device)
            model.to(device)
            transformed_img = transformed_img.to(device)
            with torch.inference_mode():
                transformed_img.to(device)
                output = model(transformed_img)
            outputp = torch.argmax(output).detach()
            outputp =outputp.item()
            result = classes[outputp]
        else:
            image_array = np.array(image)
            image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
            image_array = cv2.resize(image_array, (256, 256))
            image_tensor = torch.tensor(image_array).permute(2, 0, 1).unsqueeze(dim=0).float() / 255.0

            # Make a prediction
#             pred = model.predict(np.expand_dims(image_array, axis=0))
            # Make a prediction
            with torch.no_grad():
                output = model(image_tensor.to(device))
                outputp = torch.argmax(output).detach()
                outputp = outputp.item()
                result = labels[np.argmax(pred)]
                
#             with torch.no_grad():
#                 input_tensor = transformer(image).unsqueeze(0).to(device)
#                 outputs = model(input_tensor)
#                 _, predicted = torch.max(outputs.data, 1)
#             result = labels[predicted.item()]

        if (result != ''):
            st.success(result)
        
        search_url = f"https://www.youtube.com/results?search_query={result}"
        st.markdown(f"[YouTube]({search_url})")

if __name__=='__main__':
    main()

Overwriting app.py


In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.3.83:8501

  Stopping...
  Stopping...
